In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 128
EPISODE = 500000
TEST_EPISODE = 1000
LEARNING_RATE =1e-5
Nom = 1e-5
Weight_Deacy =1e-5
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_2 import AttributeNetwork
from my_net_2 import TripletNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(85,1024,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network = TripletNetwork(attribute_network)  #metric learning   
triplet_network.cuda(GPU)   
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
#triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE)
#
triplet_network_optim = torch.optim.Adam(triplet_network.parameters(), lr=LEARNING_RATE, 
                                         weight_decay=Weight_Deacy)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=100000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*1024

        #print(batch_features_ext)

        sample_features = attribute_network(Variable(sample_attributes).cuda(GPU).float())
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    
    batch_features_negative = np.empty_like(batch_features.numpy())
    
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    
    batch_features = Variable(batch_features).cuda(GPU).float()  # 32*2048
    
    for i in range(batch_features_negative.shape[0]):
        pick=np.random.choice(np.where(batch_labels.numpy()[i]!=train_id)[0], replace=True)
        batch_features_negative[i] = x[pick]    
    
    batch_features_negative = torch.from_numpy(batch_features_negative)
    batch_features_negative = Variable(batch_features_negative).cuda(GPU).float()  # 32*2048
    
    #batch_attributes = attribute_network(Variable(batch_attributes).cuda(GPU))
    batch_attributes = Variable(batch_attributes).cuda(GPU)
    
    dista, distb, embedded_x, embedded_y, embedded_z = triplet_network(batch_attributes, batch_features, batch_features_negative)
    
    criterion = nn.MarginRankingLoss(margin = Margin).cuda(GPU)
    
    target = torch.FloatTensor(dista.size()).fill_(-1).cuda(GPU)
    
    
    loss_triplet= criterion(dista, distb, target)
    loss_embedd = embedded_x.norm(2) + embedded_y.norm(2) + embedded_z.norm(2)
    loss = loss_triplet + Nom * loss_embedd

    # update
    triplet_network.zero_grad()
    loss.backward()
    triplet_network_optim.step()
    
    if (episode+1)%1000 == 0:
        print("episode:", episode+1, "loss", loss.data)
    if (episode+1)%1000 == 0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy = compute_accuracy(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy = compute_accuracy(test_features, test_label, np.arange(50), attributes)
        gzsl_seen_accuracy = compute_accuracy(test_seen_features, test_seen_label, np.arange(50), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        print('zsl:', zsl_accuracy)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        print('_'*100)

training...
episode: 1000 loss tensor(1.8879, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.33896218117854005
gzsl: unseen=0.0427 , seen=0.1442 , h=0.0659
____________________________________________________________________________________________________
episode: 2000 loss tensor(2.0796, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.3802990325417766
gzsl: unseen=0.0471 , seen=0.3637 , h=0.0835
____________________________________________________________________________________________________
episode: 3000 loss tensor(1.0983, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.4768689533861038
gzsl: unseen=0.0931 , seen=0.5492 , h=0.1591
____________________________________________________________________________________________________
episode: 4000 loss tensor(1.3754, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5507475813544415
gzsl: unseen=0.1481 , seen=0.6253 , h

episode: 31000 loss tensor(1.0257, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6026385224274406
gzsl: unseen=0.2887 , seen=0.6244 , h=0.3948
____________________________________________________________________________________________________
episode: 32000 loss tensor(0.7927, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6043975373790678
gzsl: unseen=0.2850 , seen=0.6240 , h=0.3913
____________________________________________________________________________________________________
episode: 33000 loss tensor(0.7033, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5980650835532102
gzsl: unseen=0.2858 , seen=0.6257 , h=0.3924
____________________________________________________________________________________________________
episode: 34000 loss tensor(0.7475, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.6026385224274406
gzsl: unseen=0.2920 , seen=0.6251 , h=0.3980
_

episode: 61000 loss tensor(0.7197, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5922603342128409
gzsl: unseen=0.3481 , seen=0.6259 , h=0.4474
____________________________________________________________________________________________________
episode: 62000 loss tensor(0.7933, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5980650835532102
gzsl: unseen=0.3448 , seen=0.6265 , h=0.4448
____________________________________________________________________________________________________
episode: 63000 loss tensor(0.8732, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5961301671064204
gzsl: unseen=0.3555 , seen=0.6248 , h=0.4532
____________________________________________________________________________________________________
episode: 64000 loss tensor(0.5869, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5996481970096745
gzsl: unseen=0.3553 , seen=0.6240 , h=0.4528
_

episode: 91000 loss tensor(0.9793, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5952506596306069
gzsl: unseen=0.3620 , seen=0.6271 , h=0.4590
____________________________________________________________________________________________________
episode: 92000 loss tensor(0.7589, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5912049252418645
gzsl: unseen=0.3671 , seen=0.6271 , h=0.4631
____________________________________________________________________________________________________
episode: 93000 loss tensor(0.6738, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5896218117854002
gzsl: unseen=0.3666 , seen=0.6273 , h=0.4627
____________________________________________________________________________________________________
episode: 94000 loss tensor(0.8644, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5938434476693052
gzsl: unseen=0.3701 , seen=0.6257 , h=0.4651
_

episode: 121000 loss tensor(0.9598, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5894459102902375
gzsl: unseen=0.3738 , seen=0.6255 , h=0.4679
____________________________________________________________________________________________________
episode: 122000 loss tensor(0.7590, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5864555848724714
gzsl: unseen=0.3669 , seen=0.6263 , h=0.4627
____________________________________________________________________________________________________
episode: 123000 loss tensor(0.7135, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5862796833773087
gzsl: unseen=0.3678 , seen=0.6244 , h=0.4629
____________________________________________________________________________________________________
episode: 124000 loss tensor(0.8504, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5866314863676341
gzsl: unseen=0.3704 , seen=0.6263 , h=0.46

...

episode: 361000 loss tensor(0.7435, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5802990325417766
gzsl: unseen=0.3601 , seen=0.6307 , h=0.4584
____________________________________________________________________________________________________
episode: 362000 loss tensor(0.8837, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5802990325417766
gzsl: unseen=0.3595 , seen=0.6309 , h=0.4580
____________________________________________________________________________________________________
episode: 363000 loss tensor(0.8888, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5797713280562885
gzsl: unseen=0.3606 , seen=0.6317 , h=0.4591
____________________________________________________________________________________________________
episode: 364000 loss tensor(0.7251, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5755496921723835
gzsl: unseen=0.3585 , seen=0.6311 , h=0.45

episode: 391000 loss tensor(1.0463, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577660510114336
gzsl: unseen=0.3599 , seen=0.6311 , h=0.4584
____________________________________________________________________________________________________
episode: 392000 loss tensor(1.2886, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5790677220756376
gzsl: unseen=0.3578 , seen=0.6315 , h=0.4568
____________________________________________________________________________________________________
episode: 393000 loss tensor(0.6593, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577660510114336
gzsl: unseen=0.3564 , seen=0.6315 , h=0.4556
____________________________________________________________________________________________________
episode: 394000 loss tensor(0.6949, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5797713280562885
gzsl: unseen=0.3590 , seen=0.6307 , h=0.4576

episode: 421000 loss tensor(0.8209, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5778364116094987
gzsl: unseen=0.3560 , seen=0.6317 , h=0.4554
____________________________________________________________________________________________________
episode: 422000 loss tensor(0.7717, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5778364116094987
gzsl: unseen=0.3580 , seen=0.6313 , h=0.4569
____________________________________________________________________________________________________
episode: 423000 loss tensor(0.6341, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577660510114336
gzsl: unseen=0.3576 , seen=0.6315 , h=0.4566
____________________________________________________________________________________________________
episode: 424000 loss tensor(0.6502, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5783641160949868
gzsl: unseen=0.3573 , seen=0.6321 , h=0.456

episode: 451000 loss tensor(0.7013, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5759014951627088
gzsl: unseen=0.3566 , seen=0.6319 , h=0.4559
____________________________________________________________________________________________________
episode: 452000 loss tensor(0.7048, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5759014951627088
gzsl: unseen=0.3573 , seen=0.6315 , h=0.4563
____________________________________________________________________________________________________
episode: 453000 loss tensor(0.4833, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5762532981530343
gzsl: unseen=0.3562 , seen=0.6321 , h=0.4556
____________________________________________________________________________________________________
episode: 454000 loss tensor(0.9107, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577660510114336
gzsl: unseen=0.3569 , seen=0.6319 , h=0.456

episode: 481000 loss tensor(0.7361, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5795954265611257
gzsl: unseen=0.3580 , seen=0.6319 , h=0.4570
____________________________________________________________________________________________________
episode: 482000 loss tensor(0.8709, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.578891820580475
gzsl: unseen=0.3581 , seen=0.6319 , h=0.4572
____________________________________________________________________________________________________
episode: 483000 loss tensor(0.5990, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.577660510114336
gzsl: unseen=0.3564 , seen=0.6327 , h=0.4559
____________________________________________________________________________________________________
episode: 484000 loss tensor(0.8238, device='cuda:0')
Testing...
class num: 10
class num: 50
class num: 50
zsl: 0.5780123131046614
gzsl: unseen=0.3583 , seen=0.6321 , h=0.4574